# Creating Heatmaps from any Sequencing Experiment

This notebook contains python code which I use to construct heatmaps comparing samples in different Sequencing Experiments. The notebook `R Code` contains the code used to construct heatmaps using the R package Pheatmap and edgeR for normalization.

**Note** All this code can also be written in R. However, I prefer using Python as much as I can, which is why I wrote this in Python.

## Creating a list of Consensus Sequences

With RNA-seq Heatmaps, each row represents a gene, making alignment of RNA-seq data to a heatmap relatively straightforward. However, with other sequencing experiments such as ATAC-seq, there are no such strict boundaries on what constitutes the rows of the heatmap. I have personally experimented with a variety of different possible ways to delimit the row in a heatmap, including tiling across the genome, and using each tile as a row, and creating a general consensus sequence of overlap between sample sequences. And I have found that using a consensus sequence provides the best results.

While there are no doubt many ways to go about creating this list, the most straightforward method that I have found is to use bedtools and a custom python script to find all regions of overlap, and save the read counts as a matrix.

### Let's get some data to play with!

I am going to download [liver](https://www.encodeproject.org/experiments/ENCSR785NEL/) and [brain](https://www.encodeproject.org/experiments/ENCSR273UFV/) BAM files from ENCODE to use as tutorial data. BAM peaks will then be called so I can get the genomic loci for each file and the read counts at each loci. There are a variety of different programs which call peaks, so use whichever method works best for you. Here are some other methods which you can use:

* [Genrich](https://github.com/jsh58/Genrich)
* [MACS2](https://github.com/taoliu/MACS)
* [HOMER](http://homer.ucsd.edu/homer/ngs/peaks.html)
* [HMMRATAC](https://github.com/LiuLabUB/HMMRATAC)

Genrich and MACS2 are the current standards, though Genrich has not been published yet. I will be using Genrich in this tutorial, but Galaxy also has a great [tutorial](https://galaxyproject.github.io/training-material/topics/epigenetics/tutorials/atac-seq/tutorial.html#peak-calling) on MACS2 if you want to use that instead.

In [1]:
import pandas as pd
import subprocess, os
from IPython.display import clear_output
import time

In [11]:
def run_genrich(bam,outpath,path=0):
    # given some bam, run genrich and send narrowPeak and bed output to output
    # path to genrich script can be specified. Otherwise, will assume that `Genrich` is the path
    if '/' in bam:
        header = bam.split('/')[-1].split('.bam')[0]
    elif '.bam' in bam:
        header = bam.split('.bam')[0]
    else:
        print("ERROR, please use a valid BAM file")
    if outpath[-1] != '/':
        outpath += '/'
        
    outnarrow = outpath + header + '.narrowPeak'
    outbed = outpath + header + '.bed'
    if path == 0:
        path = 'Genrich'
        
    subprocess.run(path+ ' -t ' + bam + ' -o ' + outnarrow + " -b " + outbed + " -r -v",shell=True,check=True)

In [ ]:
bamdir = '/mnt/labshare/chromatin-datasets/'
filenames = ['liver-p1.sorted.bam', 'liver-p2.sorted.bam','brain-p1.sorted.bam','brain-p2.sorted.bam']

# I have my bam files stored in the directory `/mnt/labshare/chromatin-datasets/`
# For the specified filenames in said directory, run genrich

for i in os.listdir(bamdir):
    if i in filenames:
        run_genrich(bamdir+i,".",'Genrich')

Now we have bed files of our peaks! 

Next, we need to get the read counts of these loci. We can do this using `bedtools coverage`. 

example:

`bedtools coverage -a liver-p1.bed -b liver-p1.sorted.bam > liver-p1.read.bed`

I recommend using pybedtools or subprocess to automate the creation of these read files.

Once we have our read files, we can work on decreasing the size of our giant bed files. The best way to do this is to use `bedtools merge`.

example:

`bedtools merge -i liver-p1.read.bed -c 5 -o sum > liver-p1.mergedread.bed`

By default there will be 5 columns in each bed file, with the fifth having the read counts. Merging BED files will compress all overlaps in a file. In the above command, we instruct bedtools to sum the contents of column 5 (the readcounts) for all overlaps that are merged. Note this method is not perfect, and that because we are using merging here, the distribution of reads will appear to be even across regions, even if there is actually higher readcount density in certain regions.

Now we have our merged files. Lets combine them all, sort them, then merge them so that we get our consensus sequences. 

`cat *.mergedread.bed | sort -k 1,1 -k2,2n | bedtools merge > consensus.bed`

### What files do we have now?

Ok, that was a lot. Lets review what files we have created so far.

1. We have bam files for every sample and experiment.
2. We have bed files and narrowPeak files for the peak calls in those experiments.
3. We have bed files with read counts per sample per experiment.
4. We have merged bed files per sample per experiment.
5. We have a consensus bed sequence.

For the creation of our heatmap, we only need the merged bed files per sample per experiment and the consensus bed sequence (4 and 5) going forward.

## Creating a read count matrix

Now that we have our consensus bed sequences, we can use it to create a matrix of read counts per loci for each sample. You can run the below code to create your matrix. However, running such code in jupyter is significantly slower than running it in terminal as a script (4 hours versus 2.5 hours). So, I suggest using the `makematrix.py` script provided, which acts essentially the same as the below function.

ex: `python makematrix.py data data/consensus.bed data/output-matrix`

In the above, we take in a directory of bed files, a consensus sequence, and specify an output for the matrix.

I recommend running this on a server. It can take anywhere from 30 minutes to 3 hours depending on the size of the data.

In [9]:
def beddir_to_matrix(bed_directory, consensus, bespec = 0):
    # Take in some directory with bed files with readconts, align to consensus and sum overlaps in same samples. Output results as a matrix
    # optionally can specify some filename header specificity to choose what bed files to read in
    
    # ex: beddir_to_matrix('data','data/consensus.bed')
    
    files = []
    headers = []
    if bed_directory[-1] != '/':
        bed_directory += '/'
    for i in os.listdir(bed_directory):
        if type(bespec) == str:
            if bespec in i:
                files.append(bed_directory+i)
                headers.append(i.split('.bed')[0])
        else:
            if '.bed' in i and consensus != i:
                files.append(bed_directory+i)
                headers.append(i.split('.bed')[0])
    
    # make dictionary
    matrixrc = {}
    matrixrc['loci'] = []
    for i in headers:
        matrixrc[i] = []
    
    # read files
    consensus = pd.read_table(consensus,sep='\t',header=None)
    
    # fill dictionary
    total = len(consensus)
    for index,row in consensus.iterrows():
        chrom = row[0]
        beg = str(row[1])
        end = str(row[2])
        matrixrc['loci'].append(chrom + ":" + beg + "-" + end)
        with open('temp','w') as out:
            out.write(chrom+'\t'+beg+'\t'+end)
        for n in range(len(files)):
            s = files[n]
            key = list(matrixrc.keys())[n+1]
            subprocess.run("intersectBed -a " + s + " -b temp -wa > intertemp",shell=True,check=True)
            try:
                inter = pd.read_table("intertemp",header=None)
                if len(inter) == 1:
                    matrixrc[key].append(inter[3].to_list()[0])
                elif len(inter) > 1:
                    matrixrc[key].append(inter[3].sum())
            except:
                matrixrc[key].append(0)
        clear_output(wait=True)
        percent = (index+1)/total * 100
        print(str(percent) + "% complete")
        print(total)
    os.remove("temp")
    os.remove("intertemp")
    pd.DataFrame.from_dict(matrixrc).to_csv('matrix',index=False)
    return(pd.DataFrame.from_dict(matrixrc))

In [ ]:
beddir_to_matrix('.','xaa')

0.3466666666666667% complete
15000
